In [ ]:
! pip install pycocotools requests tqdm
! pip install ftfy regex tqdm
! git clone https://github.com/openai/CLIP.git

import requests
from tqdm import tqdm
from pycocotools.coco import COCO
import os
import numpy as np
import torch
import torch.nn as nn
from pkg_resources import packaging
import sys
sys.path.append('/content/CLIP')
from clip import clip
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print("Available Models of CLIP : ")
print(clip.available_models())

print("Torch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())
print("Packaging version:", packaging.version.parse(torch.__version__))

In [ ]:
#%run '/content/CLIP/clip/clip.py'
model, preprocess = clip.load("ViT-B/32")
if torch.cuda.is_available():
  model.cuda().eval()
else:
  model.cpu().eval()

input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)
#print("state_dict : ", state_dict)

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!mkdir "/content/train2017"
!unzip "/content/gdrive/MyDrive/Colab Notebooks/train2017.zip" -d "/content/train2017"

Streaming output truncated to the last 5000 lines.
  inflating: /content/train2017/000000017147.jpg  
  inflating: /content/train2017/000000185201.jpg  
  inflating: /content/train2017/000000504271.jpg  
  inflating: /content/train2017/000000293002.jpg  
  inflating: /content/train2017/000000523494.jpg  
  inflating: /content/train2017/000000100095.jpg  
  inflating: /content/train2017/000000274803.jpg  
  inflating: /content/train2017/000000279428.jpg  
  inflating: /content/train2017/000000430036.jpg  
  inflating: /content/train2017/000000327198.jpg  
  inflating: /content/train2017/000000520349.jpg  
  inflating: /content/train2017/000000502828.jpg  
  inflating: /content/train2017/000000015347.jpg  
  inflating: /content/train2017/000000104028.jpg  
  inflating: /content/train2017/000000063754.jpg  
  inflating: /content/train2017/000000343413.jpg  
  inflating: /content/train2017/000000456204.jpg  
  inflating: /content/train2017/000000190806.jpg  
  inflating: /content/train2017

In [ ]:
import os
from PIL import Image
import sys
sys.path.append('/content/CLIP')
from clip import clip
images = []
original_images = []
dataset_count=0
num_training_images=10000
folder_path = "/content/train2017"
model, preprocess = clip.load("ViT-B/32")
training_dataset_split = 0.8
for filename in os.listdir(folder_path):
  if dataset_count>=num_training_images:
    break
  dataset_count+=1
  if filename.endswith(".jpg"):
    img = Image.open(os.path.join(folder_path, filename))
    rgb_img = img.convert("RGB")
    images.append(preprocess(rgb_img))
    original_images.append(rgb_img)
print(dataset_count,": Number of dataset images used in the model")

10000 : Number of dataset images used in the model


In [ ]:
state_dict = model.state_dict() or torch.load(open(clip._download(clip._MODELS["ViT-B/32"], None or os.path.expanduser("~/.cache/clip")), 'rb'), map_location="cpu")

In [ ]:
vision_width = state_dict["visual.conv1.weight"].shape[0]
vision_layers = len([k for k in state_dict.keys() if k.startswith("visual.") and k.endswith(".attn.in_proj_weight")])
vision_patch_size = state_dict["visual.conv1.weight"].shape[-1]
grid_size = round((state_dict["visual.positional_embedding"].shape[0] - 1) ** 0.5)
image_resolution = vision_patch_size * grid_size
embed_dim = state_dict["text_projection"].shape[1]
context_length = state_dict["positional_embedding"].shape[0]
vocab_size = state_dict["token_embedding.weight"].shape[0]
transformer_width = state_dict["ln_final.weight"].shape[0]
transformer_heads = transformer_width // 64
transformer_layers = len(set(k.split(".")[2] for k in state_dict if k.startswith("transformer.resblocks")))
vision_heads = vision_width // 64
print("Embeddings Dimension : ",embed_dim)
print("Image Resolution : ",image_resolution)
print("Grid Size : ",grid_size)
print("Vision Patch Size : ",vision_patch_size)
print("Vision Layers : ",vision_layers)
print("Vision Width : ",vision_width)
print("Context Length : ",context_length)
print("Vocabulary Size : ",vocab_size)
print("Transformer Width : ",transformer_width)
print("Transformer Heads : ",transformer_heads)
print("Trnsformer Layers : ",transformer_layers)
print("Vision Heads for Vision Transformer: ",vision_heads)
print(isinstance(vision_layers, (tuple, list)))

Embeddings Dimension :  512
Image Resolution :  224
Grid Size :  7
Vision Patch Size :  32
Vision Layers :  12
Vision Width :  768
Context Length :  77
Vocabulary Size :  49408
Transformer Width :  512
Transformer Heads :  8
Trnsformer Layers :  12
Vision Heads for Vision Transformer:  12
False


In [ ]:
images.append(preprocess(Image.open(os.path.join(folder_path, filename)).convert("RGB")))

In [ ]:
'''
def image_processor(folder_path, num_training_images):
  count = 0
  model, preprocess = clip.load("ViT-B/32")
  for filename in os.listdir(folder_path):
    if count >= num_training_images:
      break
    if filename.endswith(".jpg"):
      img_path = os.path.join(folder_path, filename)
      yield preprocess(Image.open(img_path).convert("RGB"))  # Yield preprocessed image
      count += 1

processed_images = image_processor(folder_path, 10000)

# Use the processed_images iterator in your subsequent code steps
for processed_image in processed_images:
  # Your code using the processed image
  pass
'''


'\ndef image_processor(folder_path, num_training_images):\n  count = 0\n  model, preprocess = clip.load("ViT-B/32")\n  for filename in os.listdir(folder_path):\n    if count >= num_training_images:\n      break\n    if filename.endswith(".jpg"):\n      img_path = os.path.join(folder_path, filename)\n      yield preprocess(Image.open(img_path).convert("RGB"))  # Yield preprocessed image\n      count += 1\n\nprocessed_images = image_processor(folder_path, 10000)\n\n# Use the processed_images iterator in your subsequent code steps\nfor processed_image in processed_images:\n  # Your code using the processed image\n  pass\n'

In [ ]:
'''
from PIL import Image
import os

# Define the folder path containing JPG images
folder_path = "/content/train2017"  # Replace with your actual folder path

# Initialize an empty list to store RGB images
images = []
original_images = []
count=0
num_training_images=5000
# !mkdir "/content/Dataset/"
# Loop through all files in the folder
for filename in os.listdir(folder_path):
  # print(f"Processed {count+1}/{num_training_images}")
  if count>=num_training_images:
    break
  count+=1
  # Check if the file is a JPG image
  if filename.endswith(".jpg"):
    # Open the image in read mode
    img = Image.open(os.path.join(folder_path, filename))

    # Convert the image to RGB format
    rgb_img = img.convert("RGB")

    # Append the RGB image to the "images" list
    images.append(preprocess(rgb_img))
    original_images.append(rgb_img)

    # !cp {folder_path+"/"+filename} {"/content/Dataset/"+filename}


# Now you have a list "images" containing all JPG images in RGB format
print(f"Total RGB images: {len(images)}")
'''

'\nfrom PIL import Image\nimport os\n\n# Define the folder path containing JPG images\nfolder_path = "/content/train2017"  # Replace with your actual folder path\n\n# Initialize an empty list to store RGB images\nimages = []\noriginal_images = []\ncount=0\nnum_training_images=5000\n# !mkdir "/content/Dataset/"\n# Loop through all files in the folder\nfor filename in os.listdir(folder_path):\n  # print(f"Processed {count+1}/{num_training_images}")\n  if count>=num_training_images:\n    break\n  count+=1\n  # Check if the file is a JPG image\n  if filename.endswith(".jpg"):\n    # Open the image in read mode\n    img = Image.open(os.path.join(folder_path, filename))\n\n    # Convert the image to RGB format\n    rgb_img = img.convert("RGB")\n\n    # Append the RGB image to the "images" list\n    images.append(preprocess(rgb_img))\n    original_images.append(rgb_img)\n\n    # !cp {folder_path+"/"+filename} {"/content/Dataset/"+filename}\n\n\n# Now you have a list "images" containing all JP

In [ ]:
if torch.cuda.is_available():
  image_input = torch.tensor(np.stack(images)).cuda()
  model.cuda()
else:
  image_input = torch.tensor(np.stack(images)).cpu()
  model.cpu()

print('Training Dataset Tensor Shape : ',image_input.shape)

Training Dataset Tensor Shape :  torch.Size([2000, 3, 224, 224])


In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    print(image_features.shape)

torch.Size([2000, 512])


In [ ]:

class ImageEmbeddingNetwork(nn.Module):
  def __init__(self):
    super(ImageEmbeddingNetwork, self).__init__()
    # Define the convolutional layers (adjust kernel sizes and padding as needed)
    self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
    self.conv2 = nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2)
    self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)

    # Define the pooling layer
    self.pool = nn.MaxPool2d(kernel_size=3, stride=2)

    # Define fully-connected layers
    self.fc1 = None  # Initialize fc1 as None here
    self.fc2 = nn.Linear(1024, 512)

    # Define activation functions (optional: ReLU is commonly used)
    self.relu = nn.ReLU()

  def forward(self, x):
    # Apply convolutional layers with activation
    x = self.relu(self.conv1(x))
    x = self.pool(self.relu(self.conv2(x)))
    x = self.pool(self.relu(self.conv3(x)))

    # Get the actual size of the output from the last convolutional layer
    num_features = x.shape[1] * x.shape[2] * x.shape[3]  # Channels * Height * Width

    # Flatten the output
    x = x.view(-1, num_features)

     # Initialize fc1 with correct input size if not initialized yet
    if self.fc1 is None:
      if torch.cuda.is_available():
        self.fc1 = nn.Linear(in_features=num_features, out_features=1024).cuda()
      else:
        self.fc1 = nn.Linear(in_features=num_features, out_features=1024).cpu()

    # Apply fully-connected layers with activation
    x = self.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [ ]:

# Define network, loss function, and optimizer
model = ImageEmbeddingNetwork()

# Move model and tensors to device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
image_input = image_input.to(device)
image_features = image_features.to(device)

criterion = nn.MSELoss()  # Mean Squared Error loss for regression-like embedding task
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate

outputs=torch.zeros(2,3)

if torch.cuda.is_available():
  outputs.cuda()
  outputs = model.cuda()(image_input[0].unsqueeze(0).cuda())  # Add batch dimension for single image
else:
  outputs.cpu()
  outputs = model(image_input[0].unsqueeze(0).cpu())  # Add batch dimension for single image


In [ ]:

# Train the network
num_epochs = 100  # Adjust number of epochs for training
num_training_loops = 10
for loop in range(num_training_loops):

  if os.path.isfile("model.pth"):
    loaded_state_dict = torch.load("/content/model.pth")
    model.load_state_dict(loaded_state_dict)
  elif os.path.isfile("/content/gdrive/MyDrive/Colab Notebooks/my_model.pth"):
    loaded_state_dict = torch.load("/content/gdrive/MyDrive/Colab Notebooks/my_model.pth", map_location=torch.device('cpu'))
    model.load_state_dict(loaded_state_dict)

  for epoch in range(num_epochs):
    #for i in range(len(image_input)):
    for i in range(20):
      # Forward pass
      if torch.cuda.is_available():
        outputs = model.cuda()(image_input[i*100:i*100+99].cuda())#.unsqueeze(0).cuda())  # Add batch dimension for single image
      else:
        outputs = model(image_input[i*100:i*100+99].cpu())#.unsqueeze(0).cpu())  # Add batch dimension for single image

      # Loss calculation
      loss = criterion(outputs, image_features[i*100:i*100+99])#.unsqueeze(0))

      # Backward pass and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # Print training progress (optional)
      if (i+1) % 1 == 0:
        print(f"Epoch: {epoch+1}/{num_epochs}, Step: {i+1}/{20}, Loss: {loss.item():.4f}")
  print(f"Completed {loop+1}/{num_training_loops} loop iteration")
  model_state_dict = model.state_dict()
  torch.save(model_state_dict, "model.pth")
  model_path = "/content/gdrive/MyDrive/Colab Notebooks/my_model.pth"
  torch.save(model.state_dict(), model_path)
print("Training complete!")

# Now you can use the trained model to extract image embeddings from new images


Streaming output truncated to the last 5000 lines.
Epoch: 51/100, Step: 5/20, Loss: 0.0107
Epoch: 51/100, Step: 6/20, Loss: 0.0113
Epoch: 51/100, Step: 7/20, Loss: 0.0137
Epoch: 51/100, Step: 8/20, Loss: 0.0134
Epoch: 51/100, Step: 9/20, Loss: 0.0123
Epoch: 51/100, Step: 10/20, Loss: 0.0114
Epoch: 51/100, Step: 11/20, Loss: 0.0151
Epoch: 51/100, Step: 12/20, Loss: 0.0146
Epoch: 51/100, Step: 13/20, Loss: 0.0143
Epoch: 51/100, Step: 14/20, Loss: 0.0136
Epoch: 51/100, Step: 15/20, Loss: 0.0143
Epoch: 51/100, Step: 16/20, Loss: 0.0139
Epoch: 51/100, Step: 17/20, Loss: 0.0156
Epoch: 51/100, Step: 18/20, Loss: 0.0154
Epoch: 51/100, Step: 19/20, Loss: 0.0148
Epoch: 51/100, Step: 20/20, Loss: 0.0150
Epoch: 52/100, Step: 1/20, Loss: 0.0110
Epoch: 52/100, Step: 2/20, Loss: 0.0116
Epoch: 52/100, Step: 3/20, Loss: 0.0120
Epoch: 52/100, Step: 4/20, Loss: 0.0108
Epoch: 52/100, Step: 5/20, Loss: 0.0102
Epoch: 52/100, Step: 6/20, Loss: 0.0108
Epoch: 52/100, Step: 7/20, Loss: 0.0133
Epoch: 52/100, Ste

In [ ]:
model_path = "/content/gdrive/MyDrive/Colab Notebooks/my_model.pth"
torch.save(model.state_dict(), model_path)

In [ ]:

print(model(image_input[0].unsqueeze(0).cuda()))

tensor([[-1.2036e-01,  5.3628e-02,  4.4086e-01,  5.3253e-01,  3.1716e-01,
         -2.4720e-01,  1.9185e-01,  1.4481e-01,  9.0623e-02,  7.6929e-02,
          1.5172e-01, -1.0628e-01,  3.5574e-02,  4.4067e-01,  3.7969e-02,
         -3.8629e-02,  5.7957e-02, -9.8279e-02, -4.1034e-03,  2.6601e-01,
          2.4352e-01,  2.8138e-02, -4.1153e-02, -5.7151e-01, -1.6084e-01,
         -2.0415e-01, -3.5339e-01,  1.6179e-01,  1.0594e-01, -4.1776e-01,
         -1.7594e-01,  3.3955e-01,  4.4632e-02,  9.4988e-02, -4.2600e-01,
          4.2450e-01, -1.2068e-01,  4.4786e-02,  1.1248e-01,  1.0622e+00,
         -7.3185e-01,  4.0613e-02, -1.7605e-01, -7.2392e-01,  1.6466e-01,
         -1.6450e+00,  4.1154e-01,  1.8473e-01,  1.0008e-01,  2.5559e-01,
          4.1574e-01,  8.4584e-04,  1.0765e-01,  1.3257e-01, -2.2892e-01,
          7.1061e-02,  4.5072e-01,  1.1196e-01, -1.6966e-01, -8.9686e-02,
         -4.4703e-01,  1.0744e-01,  1.6659e-02,  9.3970e-02, -2.0328e-01,
          1.1793e-02,  5.6275e-01,  2.

In [ ]:
print(image_features.shape)
print(image_input.shape)
#print(image_features[0])
print(image_input[0].shape)
print(image_input[8].shape)
image_features_2 = torch.zeros((2000,512))
model = ImageEmbeddingNetwork()
outputs = model.cuda()(image_input[0].unsqueeze(0).cuda())
model.load_state_dict(torch.load("/content/gdrive/MyDrive/Colab Notebooks/my_model.pth"))
torch.no_grad()
for i in range(2000):
  print(i)
  image_features_2[i] = model.cuda().forward(image_input[i].unsqueeze(0).cuda())

#image_features_2[1000:1999] = model.forward(image_input[1000:1999])
print(image_features_2.shape)

In [ ]:
criterion = nn.MSELoss()
loss = criterion(image_features.cpu(), image_features_2.cpu())
print(loss.item())

0.00013254945224616677


In [ ]:
image_features_2 /= image_features_2.norm(dim=-1, keepdim=True)


In [ ]:
image_features /= image_features.norm(dim=-1, keepdim=True)
'''text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T
print(similarity)'''

'text_features /= text_features.norm(dim=-1, keepdim=True)\nsimilarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T\nprint(similarity)'

In [ ]:
loss = criterion(image_features.cpu(), image_features_2.cpu())
print(loss.item())

0.00013254945224616677


In [ ]:
print(image_features.shape)
print(image_features_2.shape)

torch.Size([2000, 512])
torch.Size([2000, 512])


In [ ]:
similarity = image_features.cpu().numpy() @ image_features_2.cpu().detach().numpy().T
print(similarity)

[[0.9801856  0.5655094  0.70258135 ... 0.4881235  0.52928394 0.58884263]
 [0.58575255 0.97506404 0.55734    ... 0.5425698  0.71680367 0.43549022]
 [0.73251283 0.59433717 0.9770005  ... 0.5172415  0.47095838 0.45410502]
 ...
 [0.52682173 0.53548187 0.506469   ... 0.968997   0.5044687  0.4742919 ]
 [0.5519202  0.70941556 0.4688816  ... 0.5101167  0.97637737 0.454725  ]
 [0.5517439  0.5851063  0.52268606 ... 0.5149957  0.60927737 0.61121243]]


In [ ]:
sum = 0
count=0
for i in range(1000):
  count+=1
  sum+=similarity[i][i]
print(sum/count)

0.9683274517953396
